# Logit and Nested Logit

In [30]:
import pyblp
import numpy as np
import pandas as pd

pyblp.options.digits = 3
pyblp.options.verbose = False
pyblp.__version__

'0.5.0'

We will compare two simple models, the plain (IIA) logit model and the nested logit (GEV) model using the Nevo (2000) Fake Cereal Dataset

In [9]:
# First load the data
product_data = np.recfromcsv(pyblp.data.NEVO_PRODUCTS_LOCATION, encoding='utf-8')
product_data = {k: product_data[k] for k in product_data.dtype.names}

## Plain Logit

Let's start with the plain logit model under independence of irrelevant alternatives (IIA).

In this  model (indirect) utility is given by:
$$u_{ijt} = x_{jt} \beta - \alpha p_{jt} + \xi_{jt} + \varepsilon_{ijt}$$

Where $\varepsilon_{ijt}$ is distribuetd IID with Type I extreme value (Gumbel) distribution. It is common to normalize the mean utility of the outside good to zero so that $u_{i0t} = \varepsilon_{i0t}$.

This gives us aggregate marketshares 
$$s_{jt} = \frac{e^{x_{jt} \beta   - \alpha p_{jt}  + \xi_{jt}}}{\sum_k e^{x_{kt} \beta   - \alpha p_{jt}  + \xi_{kt}}}$$

If we take logs we get that:

\begin{align}
\ln s_{jt} &=x_{jt} \beta  - \alpha p_{jt}  + \xi_{jt} &- \ln\left(\sum_k e^{x_{kt} \beta  - \alpha p_{jt}  + \xi_{kt}}\right)\\
\ln s_{0t} &=  0 &- \ln\left(\sum_k e^{x_{kt} \beta   - \alpha p_{jt}  + \xi_{kt}}\right)
\end{align}

By differencing the above we get a linear estimating equation:
$$\ln s_{jt} - \ln s_{0t} = x_{jt}\beta  - \alpha p_{jt}  + \xi_{jt} $$

Because the left hand side is data, we can estimate this model using linear IV GMM.

### Implementation Details
Comparing results from the full BLP model with results from the simpler Logit model is straightforward. A Logit :class:`Problem` can be created by simply excluding the formulation for the nonlinear parameters ($X_2$) along with any agent information. 

We'll set up and solve a simpler version of the fake data cereal problem from :ref:`references:Nevo (2000)`. Since we won't include any nonlinear characteristics or parameters, we don't have to worry about configuring an optimization routine. We can still include product fixed effects using the __absorb__ option.

In [50]:
logit_formulation = pyblp.Formulation('0 + prices', absorb='C(product_ids)')
logit_formulation

prices + Absorb[C(product_ids)]

In [51]:
problem = pyblp.Problem(logit_formulation, product_data)
problem

Dimensions:
Products (N)  Markets (T)  Linear Characteristics (K1)  Nonlinear Characteristics (K2)  Cost Characteristics (K3)  Demographics (D)  Demand Instruments (MD)  Supply Instruments (MS)  Demand FEs (ED)  Supply FEs (ES)  Nesting Groups (H)
------------  -----------  ---------------------------  ------------------------------  -------------------------  ----------------  -----------------------  -----------------------  ---------------  ---------------  ------------------
    2256          94                    1                             0                             0                     0                    20                        0                    1                0                 0         

Formulations:
       Matrix Columns:            0   
------------------------------  ------
 Linear Characteristics (X1)    prices

In [52]:
results_0 = problem.solve()
results_0

Results Summary:
Cumulative  GMM   Optimization   Objective   Total Fixed Point  Total Contraction  Objective    Gradient   
Total Time  Step   Iterations   Evaluations     Iterations         Evaluations       Value    Infinity Norm
----------  ----  ------------  -----------  -----------------  -----------------  ---------  -------------
 0:00:00     2         0             1               0                  0          +4.23E+05       NA      

Linear Estimates (Robust SEs in Parentheses):
Beta:    prices   
-----  -----------
        -3.00E+01 
       (+1.01E+00)

Logit :class:`ProblemResults` can be to compute the same types of post-estimation outputs as :class:`ProblemResults` created by a full BLP problem.

## Nested Logit

We can extend the logit model to allow for correlation within a group $g$ so that:
$$u_{ijt} = x_{jt} \beta + \xi_{jt} +  \eta_{ig} +  (1-\rho) \varepsilon_{ijt}$$

Now, we require that $\eta_{ig} +  (1-\rho) \varepsilon_{ijt}$ is distributed Type I extreme value (Gumbel) distribution. As $\rho \rightarrow 1$ then all consumers stay within t the group. As $\rho \rightarrow 0$ this collapses to the IIA logit.

This gives us aggregate marketshares as the product of two logits, the within group logit and the across group logit:
$$s_{jt} = s_{j|gt} \cdot s_{gt} =  \frac{e^{(x_{jt} \beta + \xi_{jt})/(1-\rho)}}{D_g} \cdot \frac{D_g^{1-\rho}}{\sum_g D_g^{1-\sigma}} $$

With the (exponentiated) inclusive value given by: $D_g = \sum_{j \in \mathcal{J}_g} e^{(x_{jt} \beta + \xi_{jt})/(1-\sigma)}$.

After some work (see Berry (1994) or Cardell (1991)) we again obtain the linear estimating equation:
$$\ln s_{jt} - \ln s_{0t} = x_{jt}\beta - \alpha p_{jt} +\rho \ln s_{j|gt} + \xi_{jt} $$

Again, the left hand side is data, though the $\ln s_{j|gt}$ is clearly _endogenous_ which means we must instrument for it. Rather than include $\ln s_{j|gt}$ along with the _linear components_ of utility $X_1$, whenever a `nesting_id` variable is included, `pyblp` treats $\rho$ as a nonlinear $X_2$ parameter. This means that the linear component is given instead by:

$$\ln s_{jt} - \ln s_{0t} -\rho \ln s_{j|gt} = x_{jt}\beta - \alpha p_{jt}  + \xi_{jt} $$

This is done for two reasons:
- It forces the user to treat $\rho$ as an endogenous parameter.
- It extends much more easily to the RCNL model of :ref:`references:Grigolon and Verboven (2014)`.

A common choice for an additional instrument is the number of products per nest.

### Nevo Application of Nested Logit
Here it is helpful to provide the `product_data` as a `pandas` dataframe instead of as a `numpy` record array. As long as we provide `nesting_id` as a field, we don't need to change any of the formulas. We show how to construct the category groupings three different ways:
1. We put all products in a single nest (only the outside good in the other nest).
2. We put products into two nests (either Mushy or not-Mushy)
3. We put the products into three nests based on sugar content (High/Medium/Low).

We also construct an additional instrument based on the number of products per nest. Typically this is useful as a source of exogenous variation in the within group share $\ln s_{j|gt}$. However, in this example because the number of products per nest do not vary across markets, if we include product fixed effects, this instrument is irrelevant.

For all three cases we find that $\rho \approx 0.95$.

In [69]:
def solve_nl(df):
    df['demand_instruments20']=df.groupby(['market_ids','nesting_ids'])['shares'].transform(lambda x: len(x))
    problem = pyblp.Problem(logit_formulation, df)
    return problem.solve(rho=0.7)
    
# Single nest for all products (outside good in its own nest)
df=pd.DataFrame(product_data)
df['nesting_ids'] = 1
results_1 = solve_nl(df)

# Two nests for Mushy/Non-Mushy
df2=pd.DataFrame(product_data)
df2['nesting_ids'] = product_data['mushy']
results_2= solve_nl(df2)

# Three nests for low/medium/high sugar
df3=pd.DataFrame(product_data)
df3['nesting_ids']=pd.cut(df3.sugar,[-0.5,3.5,12.5,100],labels=False)
results_3= solve_nl(df3)

print("*"*25)
print("All Products in Same Nest")
print("*"*25)
print(results_1)
print("*"*25)
print("Mushy vs. non-Mushy")
print("*"*25)
print(results_2)
print("*"*25)
print("Low/Medium/High Sugar Content")
print("*"*25)
print(results_3)

*************************
All Products in Same Nest
*************************
Results Summary:
Cumulative  GMM   Optimization   Objective   Total Fixed Point  Total Contraction  Objective    Gradient   
Total Time  Step   Iterations   Evaluations     Iterations         Evaluations       Value    Infinity Norm
----------  ----  ------------  -----------  -----------------  -----------------  ---------  -------------
 0:00:00     2         0             2               0                  0          +8.08E+05    +3.19E+05  

Linear Estimates (Robust SEs in Parentheses):
Beta:    prices   
-----  -----------
        -3.05E+00 
       (+1.77E+00)

Nonlinear Estimates (Robust SEs in Parentheses):
Rho:  All Groups 
----  -----------
       +9.50E-01 
      (+5.84E-02)
*************************
Mushy vs. non-Mushy
*************************
Results Summary:
Cumulative  GMM   Optimization   Objective   Total Fixed Point  Total Contraction  Objective    Gradient   
Total Time  Step   Iterations  

### Treating within group share as exogenous
`pyblp` is designed to prevent the user from treating the within group share ($\log s_{j|g t}$) as an exogenous variable. 

In order to demonstrate why this is a bad idea, we override this feature by calculating `within_share` as an additional variable and including it in $X_1$.

We repeat the same exercise as above but note that attenuation bias causes $\rho \rightarrow 1$ and in several cases $\rho > 1$ which is not necessarily interpretable as a valid Random Utility Model (RUM). Likewise we see even though we have instrumented for price, for one of the estimates $\alpha > 0$ (demand slopes upwards).

In [68]:
within_formulation = pyblp.Formulation('0 + prices + within_share', absorb='C(product_ids)')
def solve_nl2(df):
    df['group_share']=df.groupby(['market_ids','nesting_ids'])['shares'].transform(sum)
    df['within_share']=np.log(df['shares']/df['group_share'])
    df['demand_instruments20']=df.groupby(['market_ids','nesting_ids'])['shares'].transform(lambda x: len(x))
    # make sure to drop the nesting_id's column
    problem = pyblp.Problem(within_formulation, df.drop(columns=['nesting_ids'])
    )
    return problem.solve()
    
# Single nest for all products (outside good in its own nest)
results_1 = solve_nl2(df)
# Two nests for Mushy/Non-Mushy
results_2= solve_nl2(df2)
# Three nests for low/medium/high sugar
results_3= solve_nl2(df3)

print("*"*25)
print("All Products in Same Nest")
print("*"*25)
print(results_1)
print("*"*25)
print("Mushy vs. non-Mushy")
print("*"*25)
print(results_2)
print("*"*25)
print("Low/Medium/High Sugar Content")
print("*"*25)
print(results_3)

*************************
All Products in Same Nest
*************************
Results Summary:
Cumulative  GMM   Optimization   Objective   Total Fixed Point  Total Contraction  Objective    Gradient   
Total Time  Step   Iterations   Evaluations     Iterations         Evaluations       Value    Infinity Norm
----------  ----  ------------  -----------  -----------------  -----------------  ---------  -------------
 0:00:00     2         0             1               0                  0          +6.16E+05       NA      

Linear Estimates (Robust SEs in Parentheses):
Beta:    prices     within_share
-----  -----------  ------------
        +6.88E+00    +1.29E+00  
       (+2.06E+00)  (+6.81E-02) 
*************************
Mushy vs. non-Mushy
*************************
Results Summary:
Cumulative  GMM   Optimization   Objective   Total Fixed Point  Total Contraction  Objective    Gradient   
Total Time  Step   Iterations   Evaluations     Iterations         Evaluations       Value    Inf